In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, sum as _sum, expr, regexp_replace, split, from_json, to_json
from pyspark.sql.types import StructType, StructField, StringType, ArrayType, IntegerType
from pyspark.sql import functions as f

import pandas as pd

# spark = SparkSession.builder.getOrCreate()
spark = SparkSession.builder \
    .master("local[1]") \
    .config("spark.ui.port", "4040") \
    .getOrCreate()


25/07/10 20:56:47 WARN Utils: Your hostname, RLC-Light-Trooper.local resolves to a loopback address: 127.0.0.1; using 192.168.1.10 instead (on interface en0)
25/07/10 20:56:47 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/07/10 20:56:54 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
df = spark.read.json(["/Users/ruizlorenzochavez/Downloads/steam/data/steam_data_2025-06-03.jsonl"])

In [3]:
pd_df = pd.read_json("/Users/ruizlorenzochavez/Downloads/steam/data/steam_data_2025-06-03.jsonl", lines=True)

In [4]:
df.printSchema()

root
 |-- about_the_game: string (nullable = true)
 |-- achievements: struct (nullable = true)
 |    |-- highlighted: array (nullable = true)
 |    |    |-- element: struct (containsNull = true)
 |    |    |    |-- name: string (nullable = true)
 |    |    |    |-- path: string (nullable = true)
 |    |-- total: long (nullable = true)
 |-- alternate_appid: string (nullable = true)
 |-- background: string (nullable = true)
 |-- background_raw: string (nullable = true)
 |-- capsule_image: string (nullable = true)
 |-- capsule_imagev5: string (nullable = true)
 |-- categories: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- description: string (nullable = true)
 |    |    |-- id: long (nullable = true)
 |-- content_descriptors: struct (nullable = true)
 |    |-- ids: array (nullable = true)
 |    |    |-- element: long (containsNull = true)
 |    |-- notes: string (nullable = true)
 |-- controller_support: string (nullable = true)
 |-- demos: array (

# Remove duplicates

In [5]:
df.groupBy("steam_appid").agg(f.count("*")).filter(col('count(1)')>1).show()

+-----------+--------+
|steam_appid|count(1)|
+-----------+--------+
|     223810|       2|
|     209160|       2|
|     212500|       2|
|     238210|       2|
|     221560|       2|
|     245550|       2|
|     108800|       2|
|     242840|       2|
|     412020|       2|
|     240760|       2|
|     311210|       3|
|     631780|       5|
|     288290|       5|
|     329790|       2|
|     211670|       3|
|     359550|       5|
|     244450|       2|
|     320120|       2|
|     209650|       2|
|     106010|       2|
+-----------+--------+
only showing top 20 rows



In [6]:
def find_differences_by_id(df, id_col):
    dup_ids = df.groupBy(id_col).count().filter(col("count") > 1).select(id_col)
    dup_df = df.join(dup_ids, on=id_col, how="inner")

    cols_to_check = [c for c in df.columns if c != id_col]

    diff_exprs = [
        f.collect_set(col(c)).alias(c) for c in cols_to_check
    ]

    grouped = dup_df.groupBy(id_col).agg(*diff_exprs)

    mismatch_exprs = [
        (f.size(col(c)) > 1).alias(c) for c in cols_to_check
    ]

    mismatches = grouped.select(id_col, *mismatch_exprs)

    mismatch_long = mismatches.selectExpr(
        f"`{id_col}`", 
        "stack(" + str(len(cols_to_check)) + ", " +
        ", ".join([f"'{c}', {c}" for c in cols_to_check]) +
        ") as (column, has_difference)"
    ).filter("has_difference = true")

    return mismatch_long


In [7]:
result = find_differences_by_id(df, "steam_appid")
result.show(100)

25/07/10 21:02:36 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+-----------+--------------------+--------------+
|steam_appid|              column|has_difference|
+-----------+--------------------+--------------+
|     238210|     recommendations|          true|
|     412020|     recommendations|          true|
|    2669320|     recommendations|          true|
|     311210|     recommendations|          true|
|    2195250|      about_the_game|          true|
|    2195250|          background|          true|
|    2195250|      background_raw|          true|
|    2195250|       capsule_image|          true|
|    2195250|     capsule_imagev5|          true|
|    2195250|detailed_description|          true|
|    2195250|        header_image|          true|
|    2195250|     recommendations|          true|
|    2195250|         screenshots|          true|
|     359550|     recommendations|          true|
|     244450|     recommendations|          true|
|    1426210|      about_the_game|          true|
|    1426210|          background|          true|


In [8]:
df.createOrReplaceTempView("df")

In [9]:
spark.sql("""
SELECT recommendations, ROW_NUMBER() OVER (PARTITION BY steam_appid ORDER BY recommendations DESC) AS rank FROM df WHERE steam_appid = '34330'
""").show()

+---------------+----+
|recommendations|rank|
+---------------+----+
|        {35065}|   1|
+---------------+----+



In [10]:
df = spark.sql("""
SELECT *
FROM
(SELECT *, ROW_NUMBER() OVER (PARTITION BY steam_appid ORDER BY recommendations DESC) AS rank FROM df)
WHERE rank = 1
""").drop("rank")

In [11]:
result = find_differences_by_id(df, "steam_appid")
result.show(5)

+-----------+------+--------------+
|steam_appid|column|has_difference|
+-----------+------+--------------+
+-----------+------+--------------+



# Omit columns with mostly null values

In [12]:
def get_null_counts(df):
    null_counts = df.select([
        _sum(f.when(col(c).isNull(), 1).otherwise(0)).alias(c)
        for c in df.columns
    ])
    
    result = null_counts.selectExpr(
        "stack({0}, {1}) as (column_name, null_count)".format(
            len(df.columns),
            ", ".join([f"'{c}', `{c}`" for c in df.columns])
        )
    )
    
    return result.orderBy(f.desc("null_count"))


In [13]:
null_summary = get_null_counts(df)
null_summary = null_summary.withColumn("percent", col("null_count")/df.count())
null_summary.show(100)

+--------------------+----------+-------------------+
|         column_name|null_count|            percent|
+--------------------+----------+-------------------+
|     alternate_appid|    222006| 0.9999729744338144|
|          drm_notice|    220829| 0.9946714592004036|
|ext_user_account_...|    219984| 0.9908653586292633|
|          metacritic|    217702|  0.980586634956669|
|             reviews|    209313| 0.9428003891681531|
|                 dlc|    206005| 0.9279002936778192|
|               demos|    202498| 0.9121038502423292|
|     recommendations|    201408| 0.9071942057186098|
|  controller_support|    166462|  0.749788299731546|
|        achievements|    164656|  0.741653604309677|
|        legal_notice|    152393| 0.6864178512873178|
|            fullgame|    140101| 0.6310514746950615|
|             website|    120511| 0.5428130010990397|
|      price_overview|     85779| 0.3863710069725961|
|              movies|     84831|0.38210096751526945|
|            packages|     8

In [14]:
null_columns = null_summary.filter('percent >=0.6').select('column_name').rdd.flatMap(lambda row:row).collect()

In [15]:
df = df.drop(*null_columns)

# Tidying column values

In [16]:
df.printSchema()

root
 |-- about_the_game: string (nullable = true)
 |-- background: string (nullable = true)
 |-- background_raw: string (nullable = true)
 |-- capsule_image: string (nullable = true)
 |-- capsule_imagev5: string (nullable = true)
 |-- categories: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- description: string (nullable = true)
 |    |    |-- id: long (nullable = true)
 |-- content_descriptors: struct (nullable = true)
 |    |-- ids: array (nullable = true)
 |    |    |-- element: long (containsNull = true)
 |    |-- notes: string (nullable = true)
 |-- detailed_description: string (nullable = true)
 |-- developers: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- genres: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- description: string (nullable = true)
 |    |    |-- id: string (nullable = true)
 |-- header_image: string (nullable = true)
 |-- is_free: boolean (nullable = t

In [17]:
# Flattening genre and category columns
df = df.withColumn("genre_descriptions", expr("transform(genres, x -> x.description)"))
df = df.withColumn("category_descriptions", expr("transform(categories, x -> x.description)"))

In [18]:
# Remove 'P' and ',' then cast to float
df = df.withColumns({"currency": "price_overview.currency",
                     "initial_price": regexp_replace(col("price_overview.initial_formatted"), "[P,]", "").cast("float"),
                     "final_price": regexp_replace(col("price_overview.final_formatted"), "[P,]", "").cast("float"),
                     "discount": "price_overview.discount_percent"
                     })

In [19]:
# Cleaning up language list
df = df.withColumn("languages_clean", regexp_replace(regexp_replace(col("supported_languages"), "<[^>]+>", ""), "\\*", ""))
df = df.withColumn("languages_clean",regexp_replace(col("languages_clean"), "languages with full audio support", ""))
df = df.withColumn("languages", split(col("languages_clean"), ",\\s*"))
df = df.withColumn("languages", expr("transform(languages, x -> trim(regexp_replace(x, ' - .*', '')))"))
df = df.withColumn("languages", expr("transform(languages, x -> regexp_replace(x, ' - .*', ''))"))

In [20]:
# Adding supported OS columns
df = df.withColumns({"linux": "platforms.linux",
                     "mac": "platforms.mac",
                     "pc": "platforms.windows"})

In [21]:
def clean_requirements(df, column):
    # Define schema for JSON
    schema = StructType() \
        .add("minimum", StringType()) \
        .add("recommended", StringType())

    cleaned_col = regexp_replace(
        regexp_replace(
            regexp_replace(
                regexp_replace(
                    regexp_replace(col(column), "<[^>]+>", ""),  # Remove HTML tags
                    "\\\\[nt]", ""  # Remove \n and \t
                ),
                "\\*", ""  # Remove asterisks
            ),
            "&nbsp;", ""  # Remove HTML space entities (optional)
        ),
        "\\s{2,}", " "  # Replace multiple spaces with a single space
    )

    # Add cleaned column and parse it as JSON
    df = df.withColumn(f"{column}_clean", cleaned_col)
    df = df.withColumn(f"{column}_map", from_json(col(f"{column}_clean"), schema))

    return df
    
df = clean_requirements(df, "linux_requirements")
df = clean_requirements(df, "mac_requirements")
df = clean_requirements(df, "pc_requirements")

In [22]:
# Unpacking release date
df = df.withColumns({"upcoming": "release_date.coming_soon",
                     "date_released": "release_date.date"})

In [23]:
df.select("ratings").dtypes

[('ratings', 'string')]

In [ ]:
# Reading ratings into StructType from String
ratings_schema = StructType([
    StructField("esrb", StructType([
        StructField("descriptors", ArrayType(StringType()), True),
        StructField("rating", StringType(), True),
        StructField("required_age", IntegerType(), True)
    ])),
    StructField("pegi", StructType([
        StructField("descriptors", ArrayType(StringType()), True),
        StructField("rating", StringType(), True),
        StructField("required_age", IntegerType(), True)
    ]))
])

df = df.withColumn("ratings", from_json("ratings", ratings_schema))

# PEGI & ESRB rating cleaning
def clean_ratings(df, column, new_name):
    cleaned_expr = f"""
        transform(
            {column},
            x -> regexp_replace(regexp_replace(x, '<br\\\\s*/?>', ''), '<[^>]+>', '')
        )
    """
    return df.withColumn(new_name, expr(cleaned_expr))
# ESRB
df = clean_ratings(df, "ratings.esrb.descriptors", "esrb_desc")
df = df.withColumns({"esrb_rating": "ratings.esrb.rating",
                     "esrb_age": "ratings.esrb.required_age"})

# PEGI
df = clean_ratings(df, "ratings.pegi.descriptors", "pegi_desc")
df = df.withColumns({"pegi_ratings": "ratings.pegi.rating",
                     "pegi_age": "ratings.pegi.required_age"})

In [27]:
df.explain(mode="formatted")

== Physical Plan ==
AdaptiveSparkPlan (15)
+- Project (14)
   +- Project (13)
      +- Project (12)
         +- Project (11)
            +- Project (10)
               +- Project (9)
                  +- Filter (8)
                     +- Window (7)
                        +- WindowGroupLimit (6)
                           +- Sort (5)
                              +- Exchange (4)
                                 +- WindowGroupLimit (3)
                                    +- Sort (2)
                                       +- Scan json  (1)


(1) Scan json 
Output [33]: [about_the_game#8, background#11, background_raw#12, capsule_image#13, capsule_imagev5#14, categories#15, content_descriptors#16, detailed_description#19, developers#20, genres#25, header_image#26, is_free#27, linux_requirements#29, mac_requirements#30, movies#32, name#33, package_groups#34, packages#35, pc_requirements#36, platforms#37, price_overview#38, publishers#39, ratings#40, recommendations#41, release_date#42, re

In [28]:
select_cols = ["steam_appid", 
               "name", 
               "developers", 
               "type", 
               "genre_descriptions", 
               "category_descriptions", 
               "is_free",
               "currency",
               "initial_price",
               "final_price",
               "discount",
               "languages",
               "linux",
               "linux_requirements_map",
               "mac",
               "mac_requirements_map",
               "pc",
               "pc_requirements_map",
               "upcoming",
               "date_released",
               "esrb_desc",
               "esrb_rating",
               "esrb_age",
               "pegi_desc",
               "pegi_ratings",
               "pegi_age"]
processed_df = df.select(*select_cols)
processed_df.orderBy(col("steam_appid").asc()).show()

+-----------+--------------------+--------------------+-----------+--------------------+---------------------+-------+--------+-------------+-----------+--------+--------------------+-----+----------------------+-----+--------------------+----+--------------------+--------+-------------+---------+-----------+--------+---------+------------+--------+
|steam_appid|                name|          developers|       type|  genre_descriptions|category_descriptions|is_free|currency|initial_price|final_price|discount|           languages|linux|linux_requirements_map|  mac|mac_requirements_map|  pc| pc_requirements_map|upcoming|date_released|esrb_desc|esrb_rating|esrb_age|pegi_desc|pegi_ratings|pegi_age|
+-----------+--------------------+--------------------+-----------+--------------------+---------------------+-------+--------+-------------+-----------+--------+--------------------+-----+----------------------+-----+--------------------+----+--------------------+--------+-------------+--------

# Exporting processed data to json

In [ ]:
df.coalesce(1).write.mode("append").json("/Users/ruizlorenzochavez/Downloads/steam/data/processed_json")